In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import metrics, preprocessing

import tensorflow as tf
from tensorflow.contrib import learn

# Parameters
learning_rate = 0.1
training_steps = 3000
batch_size = 128

# Network Parameters
n_input = 28  # MNIST data input (img shape: 28*28)
n_steps = 28  # timesteps
n_hidden = 128  # hidden layer num of features
n_classes = 10  # MNIST total classes (0-9 digits)

### Download and load MNIST data.
mnist = learn.datasets.load_dataset('mnist')

X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels

# It's useful to scale to ensure Stochastic Gradient Descent will do the right thing
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


def rnn_model(X, y):
  X = tf.reshape(X, [-1, n_steps, n_input])  # (batch_size, n_steps, n_input)
  # # permute n_steps and batch_size
  X = tf.transpose(X, [1, 0, 2])
  # # Reshape to prepare input to hidden activation
  X = tf.reshape(X, [-1, n_input])  # (n_steps*batch_size, n_input)
  # # Split data because rnn cell needs a list of inputs for the RNN inner loop
  X = tf.split(0, n_steps, X)  # n_steps * (batch_size, n_input)

  # Define a GRU cell with tensorflow
  lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
  # Get lstm cell output
  _, encoding = tf.nn.rnn(lstm_cell, X, dtype=tf.float32)

  return learn.models.logistic_regression(encoding, y)


classifier = learn.TensorFlowEstimator(model_fn=rnn_model, n_classes=n_classes,
                                       batch_size=batch_size,
                                       steps=training_steps,
                                       learning_rate=learning_rate)

import logging
logging.getLogger().setLevel(logging.INFO)

classifier.fit(X_train, y_train, logdir="/tmp/mnist_rnn")
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print('Accuracy: {0:f}'.format(score))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 2.65027
INFO:tensorflow:Step 101: loss = 0.24964
INFO:tensorflow:Step 201: loss = 0.151894
INFO:tensorflow:Saving checkpoints for 300 into /tmp/mnist_rnn/model.ckpt.
INFO:tensorflow:Step 301: loss = 0.0983687
INFO:tensorflow:Step 401: loss = 0.0823157
INFO:tensorflow:Step 501: loss = 0.152695
INFO:tensorflow:Saving checkpoints for 600 into /tmp/mnist_rnn/model.ckpt.
INFO:tensorflow:Step 601: loss = 0.114784
INFO:tensorflow:Step 701: loss = 0.0306895
INFO:tensorflow:Step 801: loss = 0.0774088
INFO:tensorflow:Saving checkpoints for 900 into /tmp/mnist_rnn/model.ckpt.
INFO:tensorflow:Step 901: loss = 0.11345
INFO:tensorflow:Step 1001: loss = 0.119755
INFO:tensorflow:Step 1101: loss = 0.076196
INFO:tensorflow:Saving checkpoints for 1200 into /tmp/mnist_rnn/model.ckpt.
INFO:tensorflow:Step 1201: loss = 0.0486378
INFO:tensorflow:Step 1301: loss = 0.0855077
INFO:tensorflow:Step 1401: loss = 0.0367854
INFO:tensorflow:Saving 

Accuracy: 0.982100


In [14]:
print(X_train.shape)
X_train

(55000, 784)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [15]:
X_reshaped = X_train.reshape(-1,n_steps,n_input)
print(X_reshaped.shape)
X_reshaped

(55000, 28, 28)


array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        , -0.00696077, ..., -0.00592775,
         -0.00426408,  0.        ],
        [ 0.        ,  0.        ,  0.        , ..., -0.00426406,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        , -0.00696077, ...,

In [16]:
import numpy as np

X_transpose = np.transpose(X_reshaped, (1,0,2))
print(X_transpose.shape)
X_transpose

(28, 55000, 28)


array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,

In [17]:
X_reshaped_2 = X_transpose.reshape(-1,n_input)
print(X_reshaped_2.shape)
X_reshaped_2

(1540000, 28)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [19]:
X_split = np.split(X_reshaped_2, n_steps, axis=0)
print(len(X_split))
X_split

28


[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        , -0.00426407, ..., -0.0081792 ,
          0.

In [21]:
print(X_split[0].shape)
X_split[0]

(55000, 28)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)